## Language Generation With Recurrent Neural Networks

1. Load and preprocess data
2. Tokenization and vocabulary creation
3. Pad Sequences
4. Split the data into input and target
5. Build the RNN model
6. Compile and train the model
7. Generate text

In [2]:
import tensorflow as tf
import numpy as np
import requests
import string


# Load and preprocess data

In [3]:
data = requests.get('https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = data.text.lower()
# Remove punctuation
text = text.translate(str.maketrans('', '', string.punctuation))
# Create a list of sentences
sentences = text.split('\n')

# Tokenization and vocabulary creation

In [4]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(sentences)

total_words = len(tokenizer.word_index) + 1

input_sequences = []
for sentence in sentences:
    token_list = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])


# Pad sequences

In [5]:
input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len))

# Split the data into input and target

In [12]:
input_sequences = input_sequences[:, :-1]
x_train = input_sequences[:, :-1]
y_train = input_sequences[:, -1]
y_train = tf.keras.utils.to_categorical(y_train, num_classes=total_words)


# Build the RNN model

In [24]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(x_train.shape[0], 100, input_length=max_sequence_len-1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(total_words, activation='softmax')
])


# Compile and train the model

In [25]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=100, verbose=1)


Epoch 1/100
5309/5309 [==============================] - 2070s 388ms/step - loss: 5.7117 - accuracy: 0.1875
Epoch 2/100
5309/5309 [==============================] - 2075s 391ms/step - loss: 5.3028 - accuracy: 0.2063
Epoch 3/100
5309/5309 [==============================] - 2093s 394ms/step - loss: 5.0974 - accuracy: 0.2198
Epoch 4/100
5309/5309 [==============================] - 2089s 393ms/step - loss: 4.9423 - accuracy: 0.2249
Epoch 5/100
5309/5309 [==============================] - 2091s 394ms/step - loss: 4.8168 - accuracy: 0.2299
Epoch 6/100
5309/5309 [==============================] - 2086s 393ms/step - loss: 4.7062 - accuracy: 0.2349
Epoch 7/100
5309/5309 [==============================] - 2103s 396ms/step - loss: 4.6058 - accuracy: 0.2395
Epoch 8/100
5309/5309 [==============================] - 2092s 394ms/step - loss: 4.5115 - accuracy: 0.2436
Epoch 9/100
5309/5309 [==============================] - 2091s 394ms/step - loss: 4.4232 - accuracy: 0.2478
Epoch 10/100
5309/5309 [====

## Generate text

In [ ]:
seed_text = "to be or not to"
next_words = 100

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len-1)
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)
